# Saving and Loading a Keras Model

As your models get more and more complex, you will find the time it takes to train them quickly starts to increase. They're [significantly faster using a GPU](https://www.analyticsvidhya.com/blog/2017/05/gpus-necessary-for-deep-learning/), so a brilliant option is to train a model once on the fancy OP-VR machine, save it, and then be able to load it from any machine.

## Dependencies:
* Tensorflow and keras
* [HDF5 for Python](http://docs.h5py.org/en/latest/build.html)
* numPy

## Saving:
First we'll make a model to save. This is an example model and could be swapped out. The important bits to note are that the model can be trained and compiled before saving.

In [1]:
#Load dataset from imdb. 
#This dataset is a collection of movie reviews that are marked as being either a positive or negative review.
from keras.datasets import imdb
from keras import models
from keras import layers
import numpy as np

#TOP_WORDS is the top n most frequently used words in the dataset. More uncommon words will be ignored in computations.
TOP_WORDS = 5000

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=TOP_WORDS)

#The reviews naturally have differing word-counts. 
#In order to be able to .
def vectorize_sequences(sequences, dimension=TOP_WORDS):
    #Create an all zero matrix of shape (len(sequences), dimension)
    results = np.zeros((len(sequences),dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1. #set specific indices of results[i] to 1s
    return results

#Our vectorised/padded training data
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

y_train = np.asarray(train_labels).astype('float32')
y_test =np.asarray(test_labels).astype('float32')

#Build a basic sequential model:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(TOP_WORDS,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics=['accuracy'])
model.fit(x_train, y_train, epochs=4, batch_size=512)

Using TensorFlow backend.


Epoch 1/4
25000/25000 [==============================] - 6s 230us/step - loss: 0.4626 - acc: 0.8092
Epoch 2/4
25000/25000 [==============================] - 5s 189us/step - loss: 0.2777 - acc: 0.8973
Epoch 3/4
25000/25000 [==============================] - 5s 203us/step - loss: 0.2315 - acc: 0.9130
Epoch 4/4
25000/25000 [==============================] - 5s 218us/step - loss: 0.2088 - acc: 0.9220


The model architecture is saved in a .json file, and the weights are saved separately in a .h5 file:

In [2]:
# Save the weights
model.save_weights('our_model_weights.h5')

# Save the model architecture
with open('our_model_architecture.json', 'w') as f:
    f.write(model.to_json())

You should be able to see that these files have been created in your working directory.

## Loading:
We first load the architecture from the .json file, and then fit the weights from the .h5 file.

In [3]:
from keras.models import model_from_json

# Model reconstruction from JSON file
with open('our_model_architecture.json', 'r') as f:
    loaded_model = model_from_json(f.read())

# Load weights into the new model
loaded_model.load_weights('our_model_weights.h5')

loaded_model.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics=['accuracy'])
loaded_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                80016     
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 80,305
Trainable params: 80,305
Non-trainable params: 0
_________________________________________________________________


Once compiled, we can call methods without re-training, including evaluating test data sets:

In [4]:
model.evaluate(x_test, y_test)

25000/25000 [==============================] - 6s 228us/step


[0.2896669180583954, 0.88288]